In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import math
import time

import matplotlib.pyplot as plt
import numpy as np
from IPython.display import clear_output
from matplotlib import gridspec, widgets

from robmob.robot import Robot
from robmob.rover.sensors import LDLidarSensor, RobotEspSensor
from robmob.rover.sensors import OakLiteCamera
from robmob.rover.commands import MovementFloatCommand, ResetCommand

from IPython.display import display
from ipywidgets import widgets

from robmob.robot import Robot
import pickle
import time

In [2]:
import pathlib

pathlib.Path('data/lidar/').mkdir(exist_ok=True, parents=True)
pathlib.Path('data/oak/').mkdir(exist_ok=True, parents=True)
pathlib.Path('data/imu/').mkdir(exist_ok=True, parents=True)

In [3]:
# Connexion au robot
ip_robot = 'localhost'
robot = Robot(ip_robot)
robot.connect()

In [4]:
lidar = LDLidarSensor()
oak_sensor = OakLiteCamera()
esp = RobotEspSensor()

robot.add_sensor(lidar)
robot.add_sensor(esp)

[1844301041C9600E00] [1.2.3] [0.919] [MonoCamera(0)] [error] OV7251 only supports THE_480_P/THE_400_P resolutions, defaulting to THE_480_P


In [5]:
step_id = 0
is_recording_esp = False
esp_data = []
        
def f_lidar(_):
    global step_id
    data = lidar.peek_data()
    data['timestamp'] = time.time()
    with open(f'data/lidar/lidar_{step_id:05d}.pkl', 'wb') as f:
        pickle.dump(data, f)

def f_camera(_):
    global step_id
    data = oak_sensor.peek_left()
    data = dict(data=data, timestamp=time.time())
    with open(f'data/oak/oak_{step_id:05d}.pkl', 'wb') as f:
        pickle.dump(data, f)

def f_step(_):
    global step_id
    step_id += 1

In [6]:
DEFAULT_LINEAR_SPEED = 0.1
DEFAULT_ANGULAR_SPEED = 3.14159 / 8
DEFAULT_DURATION = 1

up = widgets.Button(description='Up')
down = widgets.Button(description='Down')
left = widgets.Button(description='Left')
right = widgets.Button(description='Right')
kill = widgets.Button(description='KILL', button_style='danger')
precise_5cm = widgets.Button(description='Precise 23cm')
take_lidar = widgets.Button(description='LiDAR')
take_picture = widgets.Button(description='Camera')
next_step = widgets.Button(description='Next step')

display(widgets.HBox((left, widgets.VBox((up, down)), right)))
display(widgets.HBox((precise_5cm, take_lidar, take_picture)))
display(next_step)
display(kill)

kill.on_click(lambda _: robot.general_movement(0, 0, DEFAULT_DURATION))
up.on_click(lambda _: robot.general_movement(DEFAULT_LINEAR_SPEED, 0, DEFAULT_DURATION))
down.on_click(lambda _: robot.general_movement(-DEFAULT_LINEAR_SPEED, 0, DEFAULT_DURATION))
left.on_click(lambda _: robot.general_movement(0, DEFAULT_ANGULAR_SPEED, DEFAULT_DURATION))
right.on_click(lambda _: robot.general_movement(0, -DEFAULT_ANGULAR_SPEED, DEFAULT_DURATION))
precise_5cm.on_click(lambda _: robot.precise_movement(0.05, 0.23))
take_lidar.on_click(f_lidar)
take_picture.on_click(f_camera)
next_step.on_click(f_step)


Button(description='Next step', style=ButtonStyle())

Button(button_style='danger', description='KILL', style=ButtonStyle())